In [ ]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup, CustomJS
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

import time
import serial

useSerial = True

if useSerial == True:
    # Estalish a connection with the CD48 board
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers
    
# Set up widgets

# Create slider widget to pick desired voltage
desiredVoltageSlider = Slider(start=0, end=4.08, value=0, step=0.1, title="Ouput Voltage (V)", width = 200)

# initialize slider value
desiredVoltage = desiredVoltageSlider.value

# Add a textbox showing what the actual ouput voltage is (to compare with oscilliscope or voltmeter)
actualOutputVoltage = PreText(text='Actual Output Voltage: ' + str(desiredVoltageSlider.value), width = 200)


# define a function to be called when the voltage slider changes value
def change_voltage_output(attr, old, new):
    
    # call the global variable
    global desiredVoltage
    
    # update desired voltage to newest slider value
    desiredVoltage = desiredVoltageSlider.value
    
    # calculate the number needed for the voltage command (0-255)
    voltageNumber = round(desiredVoltage/0.016)
    
    # calculate the actual output voltage (may be different than desired due to voltage step)
    actualVoltage = round(voltageNumber*0.016, 4)
    
    # change the text displaying the actual output voltage
    actualOutputVoltage.text = 'Actual Output Voltage: ' + str(actualVoltage)
     
    # send the voltage command to the board
    voltageCommand = "V" + str(voltageNumber) + "\n"
    s.write(voltageCommand.encode())
    #voltRead = s.readline()
    
    

# when the voltage slider changes value, call function change_voltage_output
desiredVoltageSlider.on_change('value', change_voltage_output)

# create a column of widgets
voltageLayout = column([ desiredVoltageSlider, actualOutputVoltage])
        
# Add layout to current document
curdoc().add_root(voltageLayout)

# name current document
curdoc().title = "Voltage Tester"